In [1]:
import pandas as pd
import numpy as np
import duckdb

In [2]:
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

### move temporally the trusted tables into a new database

In [43]:
import os

In [44]:
con = duckdb.connect(database = "./db_test.db", read_only=False) # connection for the exploatation zone

In [45]:
for datasource in Objects:
    id = datasource['id']
    con_trust = duckdb.connect(database = f"data/trusted/db_{id}.db")
    con_trust.execute("EXPORT DATABASE './temp'")
    con_trust.close()
    con.execute(f"IMPORT DATABASE './temp'")
    #!rm -r

In [47]:
!rm ./temp/*

rm: cannot remove './temp/schools.csv': Device or resource busy


In [46]:
con.execute("SHOW TABLES").fetchall()

[('hospitals',), ('housing',), ('schools',)]

### now everything is in the explotation database

In [78]:
%%time
con.execute("""
with H as (SELECT id, price, lat, long FROM housing limit 10000),
S as (SELECT X, Y, objectid, st_grade, end_grade from schools)
SELECT
    id, price, lat, long,
    count(S.objectid) as num_schools,
    sum(if(st_grade='KG', 1, 0)) as kindergarten_schools,
FROM H left join S on (
    H.long - S.X < 0.1
    and H.long - S.X > -0.1
    and H.lat - S.Y < 0.1
    and H.lat - S.Y > -0.1
)
GROUP BY 1,2,3,4
""").fetchall()

CPU times: total: 15.3 s
Wall time: 16.3 s


[(7040933094, 875, 36.8619, -76.2738, 102, 9),
 (7039875617, 885, 30.3073, -97.6696, 139, 12),
 (7050724047, 1075, 30.2063, -97.7956, 114, 12),
 (7039870217, 1105, 30.3073, -97.6696, 139, 12),
 (7034230773, 1065, 30.2063, -97.7956, 114, 12),
 (7034106827, 1075, 30.2063, -97.7956, 114, 12),
 (7032772421, 1095, 30.2063, -97.7956, 114, 12),
 (7032794975, 1060, 30.2063, -97.7956, 114, 12),
 (7039868703, 1105, 30.3073, -97.6696, 139, 12),
 (7033586534, 1175, 30.2063, -97.7956, 114, 12),
 (7028402545, 865, 30.3073, -97.6696, 139, 12),
 (7028403120, 1155, 30.3073, -97.6696, 139, 12),
 (7047341204, 933, 29.592, -98.47, 92, 9),
 (7046850015, 775, 29.5173, -98.3217, 90, 8),
 (7034086758, 823, 29.5252, -98.342, 95, 8),
 (7050597056, 1076, 29.5855, -98.4071, 102, 9),
 (7050594538, 1230, 29.5855, -98.4071, 102, 9),
 (7047003402, 1185, 29.5252, -98.342, 95, 8),
 (7043959508, 886, 29.5252, -98.342, 95, 8),
 (7045771395, 886, 29.5252, -98.342, 95, 8),
 (7045886586, 1166, 29.5173, -98.3217, 90, 8),
 (7

In [65]:
con.execute("""
SELECT count(objectid) from schools
""").fetchall()

[(102370,)]

In [70]:
con.execute("""
SELECT count(id) from housing
""").fetchall()

[(384977,)]

In [76]:
con.execute("""
SELECT level_ from schools group by 1
""").fetchall()


[('1',), ('2',), ('N',), ('3',), ('4',)]

In [77]:
con.execute("""
SELECT st_grade, end_grade from schools group by 1,2
""").fetchall()

[('PK', '05'),
 ('06', '08'),
 ('PK', 'PK'),
 ('07', '08'),
 ('04', '05'),
 ('05', '06'),
 ('09', '12'),
 ('KG', '05'),
 ('PK', '03'),
 ('08', '12'),
 ('PK', '06'),
 ('07', '07'),
 ('10', '12'),
 ('07', '09'),
 ('03', '05'),
 ('07', '12'),
 ('KG', '06'),
 ('KG', '12'),
 ('05', '08'),
 ('PK', '08'),
 ('04', '08'),
 ('N', 'N'),
 ('KG', '08'),
 ('06', '12'),
 ('PK', '04'),
 ('KG', '09'),
 ('KG', '04'),
 ('06', '09'),
 ('01', '05'),
 ('PK', '01'),
 ('01', '12'),
 ('04', '06'),
 ('KG', '03'),
 ('PK', 'KG'),
 ('09', '13'),
 ('09', '09'),
 ('10', '10'),
 ('AE', 'AE'),
 ('03', '06'),
 ('PK', '12'),
 ('11', '12'),
 ('02', '04'),
 ('08', '09'),
 ('03', '08'),
 ('06', '06'),
 ('05', '12'),
 ('01', '04'),
 ('PK', '07'),
 ('09', '10'),
 ('04', '12'),
 ('01', '02'),
 ('09', '11'),
 ('01', '06'),
 ('KG', '01'),
 ('UG', 'UG'),
 ('02', '06'),
 ('02', '09'),
 ('04', '10'),
 ('KG', '07'),
 ('06', '07'),
 ('08', '08'),
 ('KG', '10'),
 ('01', '01'),
 ('05', '05'),
 ('01', '03'),
 ('PK', '13'),
 ('01', '07'

In [79]:
%%time
con.execute("""
with H as (SELECT id, price, lat, long FROM housing limit 10000),
S as (SELECT X, Y, objectid, beds from hospitals)
SELECT
    id, price, lat, long,
    count(S.objectid) as num_hospitals,
    sum(beds) as sum_beds,
FROM H left join S on (
    H.long - S.X < 0.1
    and H.long - S.X > -0.1
    and H.lat - S.Y < 0.1
    and H.lat - S.Y > -0.1
)
GROUP BY 1,2,3,4
""").fetchall()

CPU times: total: 3.08 s
Wall time: 1.7 s


[(7029013660, 910, 28.809, -96.9993, 6, 723.0),
 (7028237301, 934, 28.8104, -96.9742, 6, 723.0),
 (7028183894, 1279, 28.8104, -96.9742, 6, 723.0),
 (7028138517, 735, 28.8379, -97.0016, 6, 723.0),
 (7028099793, 910, 28.809, -96.9993, 6, 723.0),
 (7037905512, 1234, 28.8104, -96.9742, 6, 723.0),
 (7028482205, 800, 28.809, -96.9993, 6, 723.0),
 (7032261624, 895, 28.809, -96.9993, 6, 723.0),
 (7032252178, 1289, 28.809, -96.9993, 6, 723.0),
 (7046816231, 899, 28.809, -96.9993, 6, 723.0),
 (7050868109, 804, 28.8379, -97.0016, 6, 723.0),
 (7050586681, 1000, 28.8181, -96.9837, 6, 723.0),
 (7042884103, 950, 28.8675, -96.999, 6, 723.0),
 (7049944781, 1149, 28.809, -96.9993, 6, 723.0),
 (7045853790, 600, 28.7959, -96.9646, 6, 723.0),
 (7049898264, 1229, 28.809, -96.9993, 6, 723.0),
 (7048531255, 1149, 28.809, -96.9993, 6, 723.0),
 (7040609437, 850, 28.8417, -96.9836, 6, 723.0),
 (7030682449, 910, 28.809, -96.9993, 6, 723.0),
 (7039094787, 679, 28.8379, -97.0016, 6, 723.0),
 (7039093947, 804, 28.83